# 01. Exploring the impact of tropical cyclones on vegetation: interactive graphing and the relationship of NDVI change and cyclone windspeed

This notebook creates an interactive graph that allows the user to choose and then create Landsat and NDVI imagery of a scene before and after a tropical cyclone. The difference of NDVI before and after the tropical cyclone is then visualised. Finally, the notebook is used to import cyclone windfield data and examine the relationship between wind speed and NDVI change.

Cells titled "user requirement" indicate where users are required to edit code, the remaining code should be run normally. 

Code written in Janurary 2018 by Erin Telfer with support from Claire Krause. The notebook was completed as a graduate program project at Geoscience Australia. If you have comment or if you find an error, please contact erin.telfer@ga.gov.au. Alternatively, please contact Claire.Krause@ga.gov.au.

In [1]:
#Import libraries 

%pylab notebook

import pandas as pd
import xarray as xr
from datetime import date, timedelta
import gdal
from gdal import *

import datacube
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import matplotlib.dates
from IPython.display import display
import ipywidgets as widgets
import rasterio
import seaborn as sns
sns.set(color_codes=True)

dc = datacube.Datacube(app='dc-show changes in annual mean NDVI values')

Populating the interactive namespace from numpy and matplotlib


# User requirement: specify directory locations

In [2]:
##User input: enter the directory location of input data. Ensure "/" are used, not "\"
input_folder = '/g/data/w85/ext547/input_data/'

###User input: enter the directory location of output data. Please enter again if the same as input_folder. Ensure "/" are used, not "\"
output_folder = '/g/data/w85/ext547/cyclone_repo/output_data/'

# User requirement: specifiy location of interest and details about cyclone

In [3]:
##User input: enter area of interest

# #TamOShanter National Park
# lat_min = -17.934 #down
# lat_max = -17.863 #up
# lon_min = 146.026 #left
# lon_max = 146.096 #right

# "Debbie sugar" 
lat_min = -20.435 #down
lat_max = -20.318 #up
lon_min = 148.448 #left
lon_max = 148.629 #right

# # "Large area", bananas, Larry 
# lat_min = -18.016 #down
# lat_max = -17.830 #up
# lon_min = 145.707 #left
# lon_max = 145.873 #right

# #Mission Beach Larry
# lat_min = -17.866 #down
# lat_max = -17.856 #up
# lon_min = 146.091 #left
# lon_max = 146.109 #right


# #'Mt Dryander QLD' Debbie
# lat_min = -20.267 #down
# lat_max = -20.216 #up
# lon_min = 148.534 #left
# lon_max = 148.592 #right

# # banana Debbie
# lat_min = -20.975 #down
# lat_max = -20.967 #up
# lon_min = 148.685 #left
# lon_max = 148.692 #right

#High Mt QLD Debbie
# lat_min = -20.375 #down
# lat_max = -20.340 #up
# lon_min = 148.757 #left
# lon_max = 148.806 #right

#Debbie
# lat_min = -20.385 #down
# lat_max = -20.326 #up
# lon_min = 148.916 #left
# lon_max = 148.984 #right

##User input: enter the name of vegetation of interest, e.g. "forest" or "banana crop"
vegetation_type = 'sugar'

##User input: enter the name of the area/region/place/location of interest, e.g. "Hamilton Island"
location_name = 'Foxdale'

###User input: enter start and end date of cyclone
start_of_event= '2017-03-23'
end_of_event= '2017-04-07'

###User input: enter the name of cyclone
cyclone_name =  'Debbie'

###User input: set cloud threshold. This value defines the amount of lansdcape/cloud allowed in each scene. Scenes will not be retrieved that have less than the cloud threshold worth of image.
#The default value is "0.90" or >90% image and <10% cloud cover
cloud_free_threshold = 0.70 

# User requirement: specify the name of the windfield document (if applicable)

In [4]:
windfield_name= 'Final_corrected_TCDebbie_wind.tif' #include .extension

# Datacube query is completed

In [5]:
#Temporal range, wavelengths/band and sensors of interest are defined

#temporal range is defined
start_of_epoch = '2010-01-01'
end_of_epoch =  '2017-12-31'

#wavelengths/bands of interest are defined
bands_of_interest = [#'blue',
                     'green',
                     'red', 
                     'nir',
                     'swir1', 
                     #'swir2'
                     ]

#Landsat sensors of interest are defined
sensors = ['ls8',
       'ls7',
      'ls5'] 

#query is created
query = {'time': (start_of_epoch, end_of_epoch),}
query['x'] = (lon_min, lon_max)
query['y'] = (lat_max, lat_min)
query['crs'] = 'EPSG:4326'

print(query)

{'time': ('2010-01-01', '2017-12-31'), 'x': (148.448, 148.629), 'y': (-20.318, -20.435), 'crs': 'EPSG:4326'}


In [6]:
#Reformat variables

start_of_event=datetime.datetime.strptime(start_of_event,'%Y-%m-%d') #Convert to datetime
end_of_event=datetime.datetime.strptime(end_of_event,'%Y-%m-%d') #Convert to datetime
location_name=location_name.replace(" ","_") #replace spaces with underscore

# Data is extracted from Open Datacube

The extracted data is first filtered using the criteria in "mask_components". 
The cloudiness of the scenes is then tested, and any scenes that do not meet the given "cloud_free_threshold" are discarded.
Additionally, any pixel that is located within the ocean/sea will be converted to "nan" values with the 'land_sea' command.

In [ ]:
#Create cloud mask. This will define which pixel quality (PQ) artefacts are removed from the results. It should be noted the "land_sea" code will remove all ocean/sea pixels.

mask_components = {'cloud_acca':'no_cloud',
'cloud_shadow_acca' :'no_cloud_shadow',
'cloud_shadow_fmask' : 'no_cloud_shadow',
'cloud_fmask' :'no_cloud',
'blue_saturated' : False,
'green_saturated' : False,
'red_saturated' : False,
'nir_saturated' : False,
'swir1_saturated' : False,
'swir2_saturated' : False,
'contiguous':True,
'land_sea': 'land'}

In [ ]:
#Data for each Landsat sensor is retrieved

sensor_clean = {}

for sensor in sensors:
    #Load the NBAR and corresponding PQ
    sensor_nbar = dc.load(product= sensor+'_nbar_albers', group_by='solar_day', 
                          measurements = bands_of_interest,  **query)
    sensor_pq = dc.load(product= sensor+'_pq_albers', group_by='solar_day', 
                        fuse_func=ga_pq_fuser, **query)
    
    #Retrieve the projection information before masking/sorting
    crs = sensor_nbar.crs
    crswkt = sensor_nbar.crs.wkt
    affine = sensor_nbar.affine
    
    #Ensure there's PQ to go with the NBAR
    sensor_nbar = sensor_nbar.sel(time = sensor_pq.time)
    
    #Apply the PQ masks to the NBAR
    quality_mask = masking.make_mask(sensor_pq, **mask_components)
    good_data = quality_mask.pixelquality.loc[start_of_epoch:end_of_epoch]
    sensor_nbar2 = sensor_nbar.where(good_data)
    
    #Calculate the percentage cloud free for each scene
    cloud_free = masking.make_mask(sensor_pq, cloud_acca='no_cloud', cloud_fmask='no_cloud', 
                                   contiguous=True).pixelquality
    mostly_cloud_free = cloud_free.mean(dim=('x','y')) >= cloud_free_threshold
        
    #Discard data that does not meet the cloud_free_threshold
    mostly_good = sensor_nbar2.where(mostly_cloud_free).dropna(dim='time', how='all')
    
    mostly_good['product'] = ('time', numpy.repeat(sensor, mostly_good.time.size))    
    sensor_clean[sensor] = mostly_good

    print('loaded %s' % sensor) 
    

print ('complete')

loaded ls8


In [ ]:
#Check the output

sensor_clean

In [ ]:
#Data from different sensors are joined together and sorted so that observations are sorted by time rather than sensor
nbar_clean = xr.concat(sensor_clean.values(), 'time')
nbar_clean = nbar_clean.sortby('time')
nbar_clean.attrs['crs'] = crs
nbar_clean.attrs['affin|e'] = affine

In [ ]:
#Check that the concatenation worked

nbar_clean

# Find the scenes that occur before and after cyclone

In [ ]:
#Retun time and sensor information
product_time = nbar_clean[['time', 'product']].to_dataframe() #Add time and product to dataframe
product_time.index = product_time.index + pd.Timedelta(hours=10) #Roughly convert to local time
product_time.index = product_time.index.map(lambda t: t.strftime('%Y-%m-%d')) #Remove Hours/Minutes Seconds by formatting into a string

In [ ]:
#Save cyclone "event" date in correct format for plotting
event_dates=pd.DataFrame()
delta = end_of_event - start_of_event
for i in range(delta.days + 1):
    date=start_of_event + timedelta(days=i)
    date=pd.DataFrame([date],index=None)
    event_dates=event_dates.append(date)
event_dates.columns=['date']
event_dates=event_dates.set_index('date',drop=False)
event_dates.index=event_dates.index.map(lambda t: t.strftime('%Y-%m-%d'))
event_dates['value']=1

In [ ]:
#Save "interest dates" in correct format for plotting. Interest dates are set to help visulise avaliable scenes on either side of the cyclone.
start_of_interest= start_of_event+datetime.timedelta(days=-200) #default of 200 days
end_of_interest= end_of_event+datetime.timedelta(days=200) #default of 200 days

interest_dates=pd.DataFrame()
delta = end_of_interest - start_of_interest
for i in range(delta.days + 1):
    date=start_of_interest + timedelta(days=i)
    date=pd.DataFrame([date],index=None)
    interest_dates=interest_dates.append(date)

interest_dates.columns=['date']
interest_dates=interest_dates.set_index('date',drop=False)
interest_dates.index=interest_dates.index.map(lambda t: t.strftime('%Y-%m-%d'))
interest_dates['value']=1

In [ ]:
#Merge dates of interest with the Landsat scenes to match dates
subset_data = pd.merge(interest_dates, product_time, left_index=True,#left_on= 'date', 
                       right_index=True, how='inner') #Match dates and merge

# Create an interactive graph that shows suitable imagery before and after the cyclone

In [ ]:
#Create interactive plot that enables the user to select a scene. User should select a scene before the cyclone event

#Create widget that enables interaction with plot
w = widgets.HTML("Event information appears here when you click on the figure")
def callback(event):
    global date_int, devent
    devent = event
    date_int = event.xdata
    time_int_ = time_int.astype(datetime64[D])
    w.value = 'time_int: {}'.format(time_int)

#Set up plot
fig = plt.figure(figsize=(10,8)) #Set plot size
fig.canvas.mpl_connect('button_press_event', callback) #Plot setup
plt.title('Interactive graph: imagery avaliable before and after cyclone ' +str(cyclone_name), size=14) #Plot title
display(w) #Plot setup with interactive widget
plt.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.15) #Set border dimensions
fig.patch.set_facecolor('white') #Make border is white
fig.patch.set_alpha(0.99) #Ensure border is white

#Plot dates of interest, dates of cyclone and dates of imagery
plt.plot_date(interest_dates.date,interest_dates.value, 'g.', markersize=15, 
                            label= 'Time period of interest')
plt.plot_date(event_dates.date,event_dates.value, 'b.', markersize=15, 
                            label= 'Cyclone event')
plt.plot_date(subset_data.date,subset_data.value, 'r.', markersize=30,
                            label= 'Suitable imagery') #The term "suitable" is defined by cloud_free_threshold.
#Axis details
plt.yticks([]) #Remove y axis values as not required
plt.xticks(rotation=45,size=12) #Rotate and format size of date labels
plt.xlabel('Date', size=14) #Set X label
plt.legend(edgecolor ='none', ncol=3, loc=9, fontsize=14) #Set legend location on plot

plt.show() #Plot setup

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_1_imagery_avaliable_cyclone_'+str(cyclone_name)+'_'+str(location_name))

In [ ]:
#reformat and show date clicked on interactive graph
time_slice = matplotlib.dates.num2date(date_int).date()
subset_data['date'] = pd.to_datetime(subset_data['date'], format='%Y-%m-%d')
subset_data['difference']=(subset_data['date'] - time_slice).abs() 
time_slice=subset_data.iloc[argsort(subset_data['difference'])].date
time_slice= (list(time_slice)[0])
time_slice_str=datetime.datetime.strftime(time_slice,'%Y-%m-%d')
time_slice= np.datetime64(time_slice)
print(time_slice)

In [ ]:
#Prepare imagery
rgb = nbar_clean.sel(time =time_slice, method = 'nearest').to_array(dim='color').sel(color=[
    'swir1','nir', 'green']).transpose('y', 'x', 'color')
fake_saturation = 6000.0
rgb = rgb.astype('double')
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

In [ ]:
#Create image that shows landscape before cyclone

fig = plt.figure(figsize =(8,8)) #Edit size of plot 
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05) #Set border dimensions
fig.patch.set_facecolor('white') #Make border white 
fig.patch.set_alpha(0.99)#Ensure border white
plt.axis('off')#remove axis 
plt.title('Image 01: ' + str(vegetation_type)+ ' landscape prior (' +time_slice_str+ ') to cyclone '+ str(cyclone_name)) #add title
plt.imshow(scaled, interpolation = 'nearest') #create image

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_2_landscape_image01_'+str(cyclone_name)+'_'+str(location_name)+'_'+time_slice_str)

In [ ]:
#Calculate NDVI
ndvi = ((nbar_clean.nir-nbar_clean.red)/(nbar_clean.nir+nbar_clean.red))
ndvi = ndvi.where(ndvi>=0.0)
ndvi_of_interest= ndvi.sel(time = time_slice, method='nearest') #select scene

ndvi.attrs['crs'] = crs
ndvi.attrs['affine'] = affine

In [ ]:
#Plot NDVI for scene prior to cyclone

#Create plot
fig = plt.figure(figsize =(10,8)) #Edit size of plot 
plt.axis('off')#remove axis 
fig.patch.set_facecolor('white') #Make border white
fig.patch.set_alpha(0.99)#Ensure border white
plt.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10) #set border dimensions
ndvi_of_interest.plot(cmap = 'YlGn',vmin=0,vmax=1) #create image
plt.title('NDVI 01: ' + str(vegetation_type)+ ' landscape prior (' +time_slice_str+ ') to cyclone '+ str(cyclone_name)) #add title

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_4_NDVI01_'+str(cyclone_name)+'_'+str(location_name)+'_'+time_slice_str)

# Landscape and NDVI of scene after cyclone

In [ ]:
#Create interactive plot that enables the user to select a scene. User should select a scene after the cyclone event

#create widget that enables interaction with plot
w = widgets.HTML("Event information appears here when you click on the figure")
def callback(event):
    global date_int2, devent
    devent = event
    date_int2 = event.xdata
    time_int_ = time_int2.astype(datetime64[D])
    w.value = 'time_int: {}'.format(time_int)

#Set up plot
fig = plt.figure(figsize=(10,8)) #Set plot size
fig.canvas.mpl_connect('button_press_event', callback) #Plot setup
plt.title('Interactive graph: imagery avaliable before and after cyclone ' +str(cyclone_name), size=14) #Plot title
display(w) #Plot setup with interactive widget
plt.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.15) #Set border dimensions
fig.patch.set_facecolor('white') #Make border is white
fig.patch.set_alpha(0.99) #Ensure border is white

#Plot dates of interest, dates of cyclone and dates of imagery
plt.plot_date(interest_dates.date,interest_dates.value, 'g.', markersize=20, 
                            label= 'Time period of interest')
plt.plot_date(event_dates.date,event_dates.value, 'b.', markersize=20, 
                            label= 'Cyclone event')
plt.plot_date(subset_data.date,subset_data.value, 'r.', markersize=30,
                            label= 'Suitable imagery') #The term "suitable" is defined by cloud_free_threshold.

#Axis details
plt.yticks([]) #remove y axis values as not required
plt.xticks(rotation=45,size=12) #Rotate and format size of date labels
plt.xlabel('Date', size=14) #Set X label
plt.legend(edgecolor ='none', ncol=3, loc=9, fontsize=14) #Set legend location on plot

In [ ]:
#reformat and show date clicked on interactive graph
time_slice02 = matplotlib.dates.num2date(date_int2).date()
subset_data['date'] = pd.to_datetime(subset_data['date'], format='%Y-%m-%d')
subset_data['difference']=(subset_data['date'] - time_slice02).abs() 
time_slice02=subset_data.iloc[argsort(subset_data['difference'])].date
time_slice02= (list(time_slice02)[0])
time_slice02_str=datetime.datetime.strftime(time_slice02,'%Y-%m-%d')
time_slice02

In [ ]:
#Prepare imagery for scene after cyclone

rgb02 = nbar_clean.sel(time =time_slice02, method = 'nearest').to_array(dim='color').sel(color=[
    'swir1','nir', 'green']).transpose('y', 'x', 'color')
fake_saturation = 6000.0
rgb02 = rgb02.astype('double')
clipped_visible02 = rgb02.where(rgb02<fake_saturation).fillna(fake_saturation)
max_val02 = clipped_visible02.max(['y', 'x'])
scaled02 = (clipped_visible02 / max_val02)

In [ ]:
#Create image that shows landscape after cyclone

fig = plt.figure(figsize =(8,8)) #Edit size of plot
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05) #Set border dimensions
fig.patch.set_facecolor('white') #Make border white
fig.patch.set_alpha(0.99)#Ensure border white
plt.axis('off')#remove axis
plt.title('Image 02: ' + str(vegetation_type)+ ' landscape after ('+time_slice02_str+ ') cyclone '+ str(cyclone_name)) #add title
plt.imshow(scaled02, interpolation = 'nearest') #create image
plt.show() #show image

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_3_landscape_image02_'+str(cyclone_name)+'_'+str(location_name)+'_'+time_slice02_str) 

In [ ]:
#Select NDVI for scene after cyclone
ndvi_of_interest02= ndvi.sel(time = time_slice02, method='nearest')
ndvi_of_interest02

In [ ]:
#Plot NDVI for scene after cyclone

fig = plt.figure(figsize =(10,8)) #Edit size of plot 
plt.axis('off')#remove axis 
fig.patch.set_facecolor('white') #Make border white
fig.patch.set_alpha(0.99)#Ensure border white
plt.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10) #set border dimensions
ndvi_of_interest02.plot(cmap = 'YlGn',vmin=0,vmax=1) #create image
plt.title('NDVI 02: ' + str(vegetation_type)+ ' landscape after (' +time_slice02_str+ ') cyclone '+ str(cyclone_name)) #add title

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_5_NDVI02_'+str(cyclone_name)+'_'+str(location_name)+'_'+time_slice02_str)

# Change in vegetation before/after cyclone

In [ ]:
#Calculate the difference in NDVI from before to after cyclone

ndvi_change= ndvi_of_interest02-ndvi_of_interest
ndvi_change.attrs['affine'] = affine

In [ ]:
#Calculate the average difference in NDVI
ndvi_mean_change =ndvi_change.mean(dim=('x','y'))
percentage_ndvi_mean_change= (ndvi_mean_change /(ndvi_of_interest.mean(dim=('x','y'))))*100
print('Average difference in NDVI: ' +str(ndvi_mean_change.values)[0:5])
print('percentage change of NDVI: ' +str((percentage_ndvi_mean_change.values))[0:5]+'%')

In [ ]:
#Plot the change of NDVI
fig = plt.figure(figsize=(10,8)) #create plot
plt.axis('off')#remove axis
fig.patch.set_facecolor('white') #make border white 
fig.patch.set_alpha(0.99)#ensure border white
plt.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10) #set border dimensions
ndvi_change.plot(cmap = 'RdYlGn',vmin=-1,vmax=1) #create image

plt.title('NDVI difference for '+str(vegetation_type)+ ' landscape before and after Cyclone '+str(cyclone_name)) #add title

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_6_NDVI_change_'+str(cyclone_name)+'_'+str(location_name))

# Import and process windfield data to ensure pixel size and projection is the same as NDVI dataset

In [ ]:
# #read windfield geotiff
windfield = gdal.Open(input_folder+windfield_name, gdal.GA_ReadOnly)
windfield

In [ ]:
#get projection information from windfiled
windfield_proj = windfield.GetProjection()
windfield_proj

In [ ]:
#Convert NDVI data array to raster saved in memory

#save NDVI_change dataset to a numpy array
ndvi_array = np.asarray(ndvi_change)

#define conversion variables
cols= int(ndvi_change.x.count())
rows= int(ndvi_change.y.count())
originX= ndvi_change.affine.c
dx= ndvi_change.affine.a
originY= ndvi_change.affine.f
dy= ndvi_change.affine.e
epsg=int((str(crs)).replace('EPSG:',''))
nodata=0

#complete conversion of NDVI array into raster
driver = gdal.GetDriverByName('MEM')
ndvi_raster = driver.Create('ndvi', cols, rows, 1, gdal.GDT_Float32)
ndvi_raster.SetGeoTransform((originX, dx, 0, originY, 0, dy))
ndvi_band = ndvi_raster.GetRasterBand(1)
ndvi_band.WriteArray(ndvi_array)
ndvi_band.SetNoDataValue(nodata)
ndvi_raster_SRS = osr.SpatialReference()
ndvi_raster_SRS.ImportFromEPSG(epsg)
ndvi_raster.SetProjection(ndvi_raster_SRS.ExportToWkt())
ndvi_band.FlushCache()

In [ ]:
#Save metadata from NDVI_change
match_ds = ndvi_raster
match_proj = match_ds.GetProjection()
match_geotrans = match_ds.GetGeoTransform()
wide = match_ds.RasterXSize
high = match_ds.RasterYSize

In [ ]:
#Create a  geotiff that is the same size and is in the same projection as the NDVI dataset
drv = gdal.GetDriverByName('GTiff')
windfield_matched = drv.Create('ouput_file', wide, high, 1, gdal.GDT_Float32)
windfield_matched.SetGeoTransform(match_geotrans)
windfield_matched.SetProjection(match_proj)

#create windfield geotiff
resampling_method = gdalconst.GRA_Bilinear
gdal.ReprojectImage(windfield, windfield_matched, windfield_proj, match_proj, resampling_method)

#view datasets and projections to ensure information looks correct
print ('windfield       = ', windfield)
print ('windfield_matched             = ',windfield_matched)
print ('windfield_proj        = ',windfield_proj)
print ('match_proj      = ',match_proj)

## Save resampled windfield as an array and check array shape

In [ ]:
windfield_matched_array=windfield_matched.ReadAsArray()

#check that arrays have the same shape
print('windfield shape = '+ str(shape(windfield_matched_array)))
print('ndvi_change shape = '+ str(shape(ndvi_change.values)))

## Create a plot that examines relationship between wind and change in NDVI after a cyclone

In [ ]:
#set variables to allow automatic X-axis on plot
x_min=np.around(np.amin(windfield_matched_array),decimals=-1)-10
x_max=np.around(np.amax(windfield_matched_array),decimals=-1)+10
x_num= np.around(np.amax(windfield_matched_array)-np.amin(windfield_matched_array))

In [ ]:
plt.plot(windfield_matched_array)
plt.show()

In [ ]:
#Create an xarray array of NDVI change and windfield data
coords_da={'y': ndvi_change.y, 'x': ndvi_change.x} #create coordinate variable
windfield_da = xr.DataArray(windfield_matched_array, dims=('y','x'), coords=coords_da) #create windfield dataset in correct format
relationship_ds = xr.Dataset({'ndvi_dataset': ndvi_change, 'windfield_dataset': windfield_da}) #create xarray

relationship_xr=relationship_ds.groupby_bins(relationship_ds.windfield_dataset,bins=x_num,include_lowest=True) #groupby bins in order to understand trend
mean_relationship=relationship_xr.mean() #take mean of each bin

mean_relationship

In [ ]:
#create plot that compares change in NDVI and windfield values
fig = plt.figure(figsize=(10,8))
plt.plot(windfield_matched_array, ndvi_change.values, 'o', markeredgecolor='red', markeredgewidth=0.5, markerfacecolor='None') 
plot(mean_relationship.windfield_dataset,mean_relationship.ndvi_dataset, 'k')

plt.axis([x_min , x_max ,-1.0, 1.0], 'tight')
plt.xlabel('Modelled wind field (m s$^{-1}$)') #Set X label
plt.ylabel('Change in NDVI before and after cyclone '+str(cyclone_name)) #Set Y label
plt.plot([x_min, x_max], [0,0], 'k-', lw=1) #add blackline at 0 to plot
fig.patch.set_facecolor('white') #Make background white
fig.patch.set_alpha(0.99)#Make border white

In [ ]:
#Save figure
plt.savefig(str(output_folder)+'NB01_7_windfield_vs_NDVIchange'+str(cyclone_name)+'_'+str(location_name))